# TP2 - MongoDB - Ejercicio 2
## Francisco Javier Piqueras Martínez

Sistema Operativo: Mac OS

El Trabajo Práctico se ha implementado con la API de MongoDB para Python **pymongo**. La decisión ha sido tomada por la facilidad que brinda Jupyter Notebooks para su realizaciuón, así como la facilidad para la documentación de cada paso.

### Conexión a la Base de Datos

Creamos la conexión con la base de datos documental `ejercicio2`.

In [1]:
import pymongo
import datetime
from pymongo import MongoClient

client = MongoClient("localhost", 27017)

> Todos los apartados de este ejercicio están realizados en este notebook, la memoria solamente incluye el enunciado del ejercicio.

### Base de datos
En primer lugar, comprobamos que la base de datos `ejercicio2` y la colección `webs` se ha creado correctamente:

In [2]:
client.list_database_names()

['admin', 'config', 'ejercicio2', 'local']

In [3]:
client.ejercicio2.list_collection_names()

['webs']

In [4]:
db = client.ejercicio2

Antes de empezar, creamos una función que se encargue de mostrar los resultados por pantalla:

In [5]:
def printResult(result):
    for row in result:
        print(row)

**Consulta 1:** Número de sesiones (documentos) en los que se ha visitado la web “https://www.mikokoloko.com”. Nota: si una sesión incluye dos visitas solo se contará una vez.

In [6]:
query1 = db.webs.count_documents({'pags.url' : 'https://www.mikokoloko.com'})
query1

17

**Consulta 2:** Mostrar los distintos navegadores que se han utilizado en toda la colección.

In [7]:
query2 = db.webs.distinct('navegador')
query2

['Chrome', 'Explorer', 'firefox']

**Consulta 3:** Mostrar únicamente el campo “navegador” de las sesiones realizadas por el usuario “Bertoldo” entre las fechas “20/02/2016” y “25/02/2016” (a las 00:00:00.000 horas en ambos casos). 

In [8]:
query3 = db.webs.find({
    'nombre': 'Bertoldo', 
    '$or': [
        {'fecha' : {"$gt":  datetime.datetime(2016, 2, 20, 0, 0, 0)}},
        {'fecha' : {"$lt": datetime.datetime(2016, 2, 25, 0, 0, 0)}}
    ]
}, {'navegador': 1, '_id': 0})

printResult(query3)

{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}
{'navegador': 'firefox'}


**Agregación 1:** Obtener el número de sesiones por cada navegador, ordenadas en orden ascendente. La salida mostrará, para cada navegador, un "_id" que contiene el nombre del navegador, y un campo "total" con el número de sesiones de dicho navegador. 

In [9]:
agregation1 = db.webs.aggregate([
    {'$group': {'_id': '$navegador', 'total': {'$sum': 1}}},
    {'$sort': {'total': 1}}
])
printResult(agregation1)

{'_id': 'Chrome', 'total': 14}
{'_id': 'firefox', 'total': 17}
{'_id': 'Explorer', 'total': 20}


**Agregación 2:** Obtener la persona con mayor número de sesiones totales en la colección. La salida mostrará para cada persona (sólo 1) un "_id" conteniendo el nombre de la persona y un campo "total" con el número de sesiones totales.

In [10]:
agregation2 = db.webs.aggregate([
    {'$group': {'_id': '$nombre', 'total': {'$sum': 1}}},
    {'$sort': {'total': -1}},
    {'$limit': 1}
])
printResult(agregation2)

{'_id': 'Herminia', 'total': 20}


**Agregación 3:** Para cada persona indicar el número de URLs visitadas durante más de 100 segundos (se cuentan las repeticiones). La salida mostrará para cada persona un "_id" conteniendo el nombre de la persona y un campo "num" con el número de URLs. Pista: utilizar la opción $unwind de aggregate.

In [11]:
agregation3 = db.webs.aggregate([
    {'$unwind': '$pags'},
    {'$match': {'pags.segs': {'$gt': 100}}},
    {'$group': {'_id': '$nombre', 'num': {'$sum': 1}}}
])
printResult(agregation3)

{'_id': 'Aniceto', 'num': 35}
{'_id': 'Herminia', 'num': 40}
{'_id': 'Bertoldo', 'num': 51}


**Agregación 4:** Nombre de la persona que visita más páginas de media por sesión. La salida mostrará un "_id" conteniendo el nombre de la persona y un campo "media" con el valor de la media de páginas visitadas. Pistas: calcular el número de URLs para cada sesión y luego calcular la media de ese dato para cada persona / No hace falta utilizar $unwind. 

In [12]:
agregation4 = db.webs.aggregate([
    {'$project': {'_id': 0, 'nombre': '$nombre', 'media': {'$size': '$pags'}}},
    {'$group': {'_id': '$nombre', 'media': {'$avg': '$media'}}},
    {'$sort': {'media': -1}},
    {'$limit': 1}
])
printResult(agregation4)

{'_id': 'Bertoldo', 'media': 4.411764705882353}


**Agregación 5:** Calcular el total de sesiones con el navegador "Explorer" en 2016. La salida mostrará el "_id" conteniendo el año (2016) y un campo "total" con el número total de sesiones. Pista: para obtener el año de una fecha se puede utilizar `{$year:"$fecha"}`


In [41]:
agregation5 = db.webs.aggregate([
    {'$project': {'_id': 0, 'navegador': '$navegador', 'fecha': {'$year': '$fecha'}}},
    {'$match': {'$and':[{'fecha': 2016}, {'navegador': 'Explorer'}]}},
    {'$group': {'_id': '$fecha', 'total': {'$sum': 1}}}
])
printResult(agregation5)

{'_id': 2016, 'total': 10}


**Agregación 6:**  Para cada persona calcular el tiempo acumulado de todas las URLs visitadas, en minutos. La salida mostrará un campo “nombre” con el nombre de la persona, y un campo “tiempomin” con el tiempo acumulado. La salida estará́ ordenada alfabéticamente por los nombres de los usuarios. Pista: utilizar la opción $unwind de aggregate.

In [33]:
agregation6 = db.webs.aggregate([
    {'$unwind': '$pags'},
    {'$project': {'_id': 0, 'nombre': '$nombre', 'tiempo': {'$divide': [ '$pags.segs', 60 ]}}},
    {'$group': {'_id': '$nombre', 'tiempomin': {'$sum': '$tiempo'}}},
    {'$project': {'_id':0, 'nombre': '$_id', 'tiempomin': '$tiempomin'}},
    {'$sort': {'nombre': 1}}
])
printResult(agregation6)

{'nombre': 'Aniceto', 'tiempomin': 329.35}
{'nombre': 'Bertoldo', 'tiempomin': 263.3333333333333}
{'nombre': 'Herminia', 'tiempomin': 551.6666666666666}


**Agregación 7:** Obtener el nombre de las personas que han visitado alguna página que comience por “https”. La salida mostrará para cada persona únicamente el campo “nombre” que contendrá su nombre. Pista: utilizar el operador de proyección $substr

In [46]:
agregation7 = db.webs.aggregate([
    {'$unwind': '$pags'},
    {'$project': {'nombre': '$nombre', 'url': {'$substr': ['$pags.url', 0, 5]}}},
    {'$match': {'url': 'https'}},
    {'$group': {'_id': '$nombre'}},
    {'$project': {'nombre':'$_id', '_id':0}}
])
printResult(agregation7)

{'nombre': 'Aniceto'}
{'nombre': 'Bertoldo'}
